In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import pickle

perlis = "https://www.bernas.com.my/contact-us/rice-mills/kbb-perlis"
kedah = "https://www.bernas.com.my/contact-us/rice-mills/kbb-kedah"
penang = "https://www.bernas.com.my/contact-us/rice-mills/kbb-penang"
perak = "https://www.bernas.com.my/contact-us/rice-mills/kbb-perak"
selangor = "https://www.bernas.com.my/contact-us/rice-mills/kbb-selangor"
pahang = "https://www.bernas.com.my/contact-us/rice-mills/kbb-pahang"
states = [perlis,kedah,penang,perak,selangor,pahang]
sts = ["Perlis","Kedah","Penang","Perak","Selangor","Pahang"]
div_xp = '//*[@id="section-id-1662608736593"]/div/div'


In [2]:
factories = []
for st,state in zip(sts,states):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920x1080")
    driver = webdriver.Chrome(service=Service("../webdriver/chromedriver.exe"),options=chrome_options)
    driver.get(state)
    div = WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,div_xp)))
    txt = div.text.split("\n")
    driver.close()

    heads = [txt.index(line) for line in txt if "KBB" in line]

    for head in heads:
        dic = {
        "state":st,
        "name":txt[head],
        "address":txt[head+1].replace("Address: ",""),
        "phone":txt[head+2].replace("Phone: ",""),
        "fax":txt[head+3].replace("Fax: ",""),
        "latdms":txt[head+4].replace("GPS: ","").split(" ")[0],
        "longdms":txt[head+4].replace("GPS: ","").split(" ")[1],
        "latitude":txt[head+5].split(",")[0],
        "longitude":txt[head+5].split(",")[1]
        }
        factories.append(dic)

In [3]:
df = pd.DataFrame(factories)
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df

,state,name,address,phone,fax,latdms,longdms,latitude,longitude
0,Perlis,KBB Utan Aji,"Lot 2074, 01000 Kangar, Perlis , Malaysia.",+604 976 1133,+604 976 5155,"6°25'40.1""N","100°11'45.9""E",6.427811,100.196069
1,Perlis,KBB Kuala Perlis,"Lot PT 136, 02000 Kuala Perlis, Perlis, Malaysia.",+604 985 4133,+604 985 5487,"6°22'27.3""N","100°09'27.0""E",6.374250,100.157500
2,Perlis,KBB Simpang Empat,"Lot 236 & 237 , Simpang Empat, Jalan T.Tulang ...",+604 980 7244,+604 980 7191,"6°20'41.7""N","100°11'58.0""E",6.344905,100.199449
3,Kedah,KBB Kodiang,"Lot 3346, Mukim Kepelu, 06100 Kodiang, Kedah, ...",+604 925 5346,+604 925 2877,"6°23'15.1""N","100°18'15.1""E",6.387537,100.304194
4,Kedah,KBB Megat Dewa,"Lot PT 437, 06100 Alor Setar, Kedah, Malaysia.",+604 925 1349,+604 925 1679,"6°19'42.0""N","100°20'13.2""E",6.328342,100.336998
5,Kedah,KBB Sungai Baru,"Lot 615 & 653 , 06250 Alor Setar, Kedah, Malay...",+604 733 0896,+604 733 2173,"6°08'54.1""N","100°19'21.1""E",6.148350,100.322520
6,Kedah,KBB Bukit Raya,"Lot PT 28, 06700 Pendang, Kedah, Malaysia.",+604 759 6227,+604 759 7429,"6°01'11.5""N","100°26'49.8""E",6.019854,100.447156
7,Kedah,KBB Pering,"PT 668, Mukim Pering, Jalan Sanglang , 06000 K...",+604 925 2673,+604 925 3526,"6°20'10.9""N","100°16'10.6""E",6.336360,100.269600
8,Kedah,KBB Kangkong,"Lot PT 2 & 740 Mukim Kangkong , 06650 Alor Set...",+604 764 1278,+604 764 2137,"6°01'17.4""N","100°22'06.0""E",6.021492,100.368331
9,Kedah,KBB Teluk Kechai,"Lot 625, Mukim Teluk Kechai , 06600 Kuala Keda...",+604 762 1844,+604 762 1103,"6°05'36.1""N","100°18'23.9""E",6.093353,100.306637


# Persist

In [ ]:
with open('serialized/df_bernasmills.pkl','wb') as file:
    pickle.dump(df,file)

# Reload

In [ ]:
with open('serialized/df_bernasmills.pkl','rb') as file:
    df = pickle.load(file)

In [ ]:
# export df
df.to_csv("data/bernasmill_locations.csv",index=False)

# Visual QC

In [ ]:
from geopandas import GeoDataFrame
from shapely.geometry import Point
import matplotlib.pyplot as plt
import contextily as cx

geometry = [Point(xy) for xy in zip(df.longitude,df.latitude)]
gdf = GeoDataFrame(df,crs="EPSG:4326",geometry=geometry)
gdf = gdf.to_crs(epsg=3857)
gdf['coords'] = gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
ax = gdf.plot(figsize=(20,10),c='red')
cx.add_basemap(ax, source=cx.providers.OpenTopoMap)
# cx.add_basemap(ax, source=cx.providers.Stamen.TonerLabels)
ax.set_axis_off()
# for idx, row in gdf.iterrows():
#     plt.annotate(text=row['name'],xy=row['coords'][0])
